Stage 3: Python  notebook: Demonstrate simple queries in Python notebooks over your data. In particular, apply sentiment analysis to data in different dates and periods and generate simple plots. Enable search queries over the data.

In [ ]:
# Analyze URV, lithops and other entities related to this project (ibm?)
# spain, catalonia...

# TODO: Use file chunking to read the csv file generated in Stage 2 - Preprocessing.
# https://github.com/lithops-cloud/lithops/blob/master/docs/file_chunking.md

In [ ]:
import pandas as pd

pd.set_option("display.precision", 2)

In [ ]:
df = pd.read_csv("input.csv")
df.head()